In [1]:
import dill
from main.manipulation import Manipulation
from main.data_processing import evaluate_profile, create_graph_from_matrix
import pandas as pd
from main.manipulation_utils import get_children_generation_options, check_transitivity, one_cost_children_generation, useful_change
from IPython.display import display
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
# examples.values[0]: (current_prefs[voters[preference]], true_prefs[voters[preference]])
examples = {
    1: (
        (pd.DataFrame({0: [0,1,1], 1: [-1,0,1], 2:[-1,-1,0]}), pd.DataFrame({0: [0,-1,0], 1: [1,0,1], 2:[0,-1,0]})),
        (pd.DataFrame({0: [0,1,1], 1: [-1,0,1], 2:[-1,-1,0]}), pd.DataFrame({0: [0,0,0], 1: [0,0,-1], 2:[0,1,0]}))
    ),
    2: (
        (pd.DataFrame({0: [0,0,0], 1: [0,0,0], 2:[0,0,0]}), pd.DataFrame({0: [0,0,0], 1: [0,0,1], 2:[0,-1,0]})),
        (pd.DataFrame({0: [0,0,0], 1: [0,0,0], 2:[0,0,0]}),pd.DataFrame({0: [0,0,0], 1: [0,0,-1], 2:[0,1,0]}))
    ),
    3: (
        (pd.DataFrame({0: [0,0,0,0], 1: [0,0,0,-1], 2:[0,0,0,-1], 3: [0,1,1,0]}), pd.DataFrame({0: [0,0,0,0], 1: [0,0,0,0], 2:[0,0,0,0], 3: [0,0,0,0]})),
        (pd.DataFrame({0: [0,1,1,1], 1: [-1,0,1,1], 2:[-1,-1,0,1], 3: [-1,-1,-1,0]}), pd.DataFrame({0: [0,0,0,0], 1: [0,0,0,0], 2:[0,0,0,0], 3: [0,0,0,0]}))
    ),
    4: (
        (pd.DataFrame({0: [0,-1,-1], 1: [1,0,0], 2:[1,0,0]}), pd.DataFrame({0: [0,-1,1], 1: [1,0,1], 2:[-1,-1,0]}), pd.DataFrame({0: [0,1,1], 1: [-1,0,1], 2:[-1,-1,0]})),
        (pd.DataFrame({0: [0,-1,-1], 1: [1,0,0], 2:[1,0,0]}), pd.DataFrame({0: [0,-1,1], 1: [1,0,1], 2:[-1,-1,0]}), pd.DataFrame({0: [0,1,1], 1: [-1,0,1], 2:[-1,-1,0]}))
    )
}


In [28]:
for p in examples[4][0]:
    display(p)

,0,1,2
0,0,1,1
1,-1,0,0
2,-1,0,0


,0,1,2
0,0,1,-1
1,-1,0,-1
2,1,1,0


,0,1,2
0,0,-1,-1
1,1,0,-1
2,1,1,0


In [29]:
for p in examples[4][1]:
    display(p)

,0,1,2
0,0,1,1
1,-1,0,0
2,-1,0,0


,0,1,2
0,0,1,-1
1,-1,0,-1
2,1,1,0


,0,1,2
0,0,-1,-1
1,1,0,-1
2,1,1,0


In [26]:
def test(example_to_test,
         manipulation_voter,
         k,
         rule,
         do_omissions,
         do_additions,
         do_flips,
         expected_winner,
         expected_profiles,
         verbose = False
        ):
    alph_order = {}
    for i in example_to_test[0][0].index:
        alph_order[i] = i
    
    winner, possible_winners, scores_of_alternatives = evaluate_profile(graphs=example_to_test[0], k=k, method=rule, alphabetical_order=alph_order)
    
    man = Manipulation(
    all_preferences=example_to_test[0],
    preference_idx=manipulation_voter,
    winner=winner,
    truthful_profile=example_to_test[1],
    possible_winners=possible_winners,
    scores_of_alternatives=scores_of_alternatives,
    alphabetical_order_of_alternatives=alph_order,
    method=rule,
    k=k,
    do_additions=do_additions,
    do_omissions=do_omissions,
    do_flips=do_flips,
    verbose=verbose,
    hard_exit_time_limit=300
    )

    result = man.manipulation_move()
    if result is not None:
        new_prof, winner = result
        for p in new_prof:
            display(p)
    else:
        assert expected_profiles is None
        assert expected_winner is None
        return 
    
    assert winner == expected_winner
    profile_check_pass = False
    for p in expected_profiles:
        if new_prof[manipulation_voter].equals(p):
            profile_check_pass = True
    assert profile_check_pass

## ex. 1

Manipulating voter: voter_1

Rule: Veto, k=1

Order of alternatives: 0: a_0, 1:a_1, 2: a_2

- Omission_allowed: yes, Addition_allowed: yes, Flips_allowed: yes . Result:

Alternative 1 is preferred. Manipulation happened.
New preference of voter_1:  [[0,0,0],[0,0,0],[0,0,0]]

- Omission_allowed: no, Addition_allowed: yes, Flips_allowed: yes . Result:

Alternative 1 is preferred. Manipulation happened.
New preference of voter_1:  [[0,1,1],[-1,0,1],[-1,-1,0]] or  [[0,-1,-1],[1,0,-1],[1,1,0]]

- Omission_allowed: no, Addition_allowed: no, Flips_allowed: yes . Result:

No manipulation possible

In [11]:
exp_winner = 1
exp_profs = [pd.DataFrame({0: [0,0,0], 1: [0,0,0], 2:[0,0,0]})]
test(examples[1], 1, 1, 'veto', True, True, True, exp_winner, exp_profs, verbose=False)

In [12]:
exp_winner = 1
exp_profs = [pd.DataFrame({0: [0,-1,-1], 1: [1,0,-1], 2:[0,1,0]}), pd.DataFrame({0: [0,1,1], 1: [-1,0,1], 2:[-1,-1,0]}), pd.DataFrame({0: [0,-1,-1], 1: [1,0,-1], 2:[1,1,0]})]
test(examples[1], 1, 1, 'veto', False, True, True, exp_winner, exp_profs)

In [13]:
test(examples[1], 1, 1, 'veto', False, False, True, None, None)

## ex. 2

Manipulating voter: voter_1

Rule: Veto, k=1

Order of alternatives: 0: a_0, 1:a_1, 2: a_2

- Omission_allowed: yes, Addition_allowed: no, Flips_allowed: yes . Result:

Alternative 1 is preferred. Manipulation happened.
New preference of voter_1: [[0,0,0],[0,0,-1],[0,1,0]]


- Omission_allowed: yes, Addition_allowed: yes, Flips_allowed: no. Result:

Alternative 1 is preferred. Manipulation happened.
New preference of voter_1: [[0,1,1],[-1,0,1],[-1,-1,0]]

- Omission_allowed: yes, Addition_allowed: yes, Flips_allowed: yes. Result:

Alternative 1 is preferred. Manipulation happened.
New preference of voter_1: [[0,1,1],[-1,0,1],[-1,-1,0]] or [[0,0,0],[0,0,-1],[0,1,0]]

- Omission_allowed: yes, Addition_allowed: no, Flips_allowed: no. Result:
No manipulation possible

In [14]:
exp_winner = 1
exp_profs = [pd.DataFrame({0: [0,0,0], 1: [0,0,-1], 2:[0,1,0]})]
test(examples[2], 1, 1, 'veto', True, False, True, exp_winner, exp_profs)

In [16]:
exp_winner = 1
exp_profs = [pd.DataFrame({0: [0,1,1], 1: [-1,0,1], 2:[-1,-1,0]})]
test(examples[2], 1, 1, 'veto', True, True, False, exp_winner, exp_profs)

In [19]:
exp_winner = 1
exp_profs = [pd.DataFrame({0: [0,1,1], 1: [-1,0,1], 2:[-1,-1,0]}), pd.DataFrame({0: [0,0,0], 1: [0,0,-1], 2:[0,1,0]})]
test(examples[2], 1, 1, 'veto', True, True, True, exp_winner, exp_profs)

In [18]:
test(examples[2], 1, 1, 'veto', True, False, False, None, None)

## ex. 3
approval
k=1
voter=0
- All allowed: Manipulation happened.  alternative 3 is preferred. 

- No addition allowed: No manipulation possible.

In [ ]:
exp_winner = 3
exp_profs = [pd.DataFrame({0: [0,1,1], 1: [-1,0,1], 2:[-1,-1,0]}), pd.DataFrame({0: [0,0,0], 1: [0,0,-1], 2:[0,1,0]})]
test(examples[3], 0, 1, 'approval', True, True, True, exp_winner, exp_profs, verbose=True)

Starting manipulation move
investigating alternative 3
p_score=0, winner_score=1
Alternative 3 is preferred. Investigating possible manipulation.
in tree_generation_level_1_onwards
in examine_matrices_cost_2
all generated matrices so far 16
in examine_matrices_cost_1
all generated matrices so far 16
new_max_cost_so_far: 2
iteration took 0.9177651405334473 sec.
in examine_matrices_cost_1
all generated matrices so far 208
in examine_matrices_cost_2
all generated matrices so far 208
new_max_cost_so_far: 3
iteration took 24.886722087860107 sec.
in examine_matrices_cost_2
all generated matrices so far 2257
in examine_matrices_cost_1


In [22]:
test(examples[3], 0, 1, 'approval', True, False, True, None, None)

## ex. 4
approval
k=2
voter=0
- All allowed: Manipulation happened.  New preference of voter_0: [[0,-1,-1],[1,0,-1],[1,1,0]]
- No addition allowed: No manipulation possible.

In [19]:
exp_winner = 0
exp_profs = [pd.DataFrame({0: [0,-1,-1], 1: [1,0,-1], 2:[1,1,0]})]
test(examples[4], 0, 2, 'approval', True, True, True, exp_winner, exp_profs)

In [22]:
test(examples[4], 0, 2, 'approval', True, False, True, None, None)